Of course. Let's begin with the first algorithm on our list: Factoring.

***

### 1. Factoring (Shor's Algorithm)

Shor's algorithm is a quantum algorithm for integer factorization, discovered by Peter Shor in 1994. It is arguably the most famous quantum algorithm due to its profound implications for cryptography.

* **Complexity**: Superpolynomial Speedup
    * **Quantum**: The algorithm runs in polynomial time, specifically in $O((\log N)^2 (\log \log N)(\log \log \log N))$ time on a quantum computer, where $N$ is the integer to be factored [82, 125].
    * **Classical**: The best-known classical algorithm is the general number field sieve, which runs in sub-exponential time, approximately $exp(O((\log N)^{1/3} (\log \log N)^{2/3}))$. For large $N$, this is significantly slower than the quantum algorithm.

* **Implementation Libraries**:
    * **Cirq**: Examples of Shor's algorithm are available in the Cirq library, demonstrating how to construct the quantum circuits for the order-finding subroutine. You can find these within the `examples/` directory of the [Cirq GitHub repository](https://github.com/quantumlib/Cirq/tree/main/examples).
    * **Classiq**: The Classiq platform provides a higher-level, functional approach to building quantum circuits. Their library contains implementations of Shor's algorithm that abstract away much of the gate-level complexity. You can explore these in the `algorithms/` section of the [Classiq library on GitHub](https://github.com/Classiq/classiq-library/tree/main/algorithms).
    * Other libraries like **PennyLane** and **Qrisp** also have implementations available.

---

### **Detailed Theory**

Shor's algorithm cleverly transforms the hard problem of factoring an integer into the problem of finding the period of a function, which is a task a quantum computer can perform efficiently.

The overall structure of the algorithm is as follows:

**Part 1: Classical Reduction to Order-Finding**

Let $N$ be the large composite integer we wish to factor.

1.  **Initial Check**: Perform a classical check to see if $N$ is even or a prime power. If $N$ is even, 2 is a factor. If $N=p^k$ for some prime $p$, we can find $p$ classically.

2.  **Pick a Random Number**: Choose a random integer $a$ such that $1 < a < N$.

3.  **Compute GCD**: Calculate the greatest common divisor, $gcd(a, N)$. If $gcd(a, N) \neq 1$, we have found a non-trivial factor of $N$, and we are done.

4.  **The Order-Finding Problem**: If $gcd(a, N) = 1$, we now need to find the **order** (or period) $r$ of $a$ modulo $N$. The order $r$ is the smallest positive integer such that:
    $a^r \equiv 1 \pmod{N}$
    This is the computationally hard part that the quantum computer will solve.

5.  **Finding the Factors**: Once the quantum subroutine returns the order $r$, we check two conditions:
    * Is $r$ even?
    * Is $a^{r/2} \not\equiv -1 \pmod{N}$?

    If both conditions are met, then we can find a non-trivial factor of $N$ by computing:
    $p = gcd(a^{r/2} - 1, N)$ and $q = gcd(a^{r/2} + 1, N)$
    This works because if $a^r \equiv 1 \pmod{N}$, then $(a^{r/2} - 1)(a^{r/2} + 1) \equiv 0 \pmod{N}$. This means $N$ divides the product on the left, and if the conditions are met, neither term is itself a multiple of $N$, so they must share non-trivial factors with $N$. If the conditions are not met, we go back to step 2 and pick a new random number $a$. The probability of success is high for a random choice of $a$.

**Part 2: The Quantum Order-Finding Subroutine**

This is the core of Shor's algorithm. It uses the **Quantum Fourier Transform (QFT)** to find the period $r$ of the function $f(x) = a^x \pmod{N}$.

1.  **State Preparation**: We start with two quantum registers. Let's say the first register has $n$ qubits, where $2^n$ is large enough to store the period $r$ (typically $n$ is chosen such that $N^2 \le 2^n < 2N^2$). The second register must have enough qubits to store a number up to $N-1$. We initialize the system in the state:
    $|\psi_0\rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} |x\rangle |0\rangle$
    This is done by applying a Hadamard gate to each qubit in the first register.

2.  **Modular Exponentiation**: We apply a quantum operator that performs modular exponentiation, mapping the initial state to:
    $|\psi_1\rangle = \frac{1}{\sqrt{2^n}} \sum_{x=0}^{2^n-1} |x\rangle |a^x \pmod{N}\rangle$
    This operation entangles the two registers.

3.  **Measurement of the Second Register**: We now measure the second register. Let the result of this measurement be some value $k$. Due to the periodic nature of $f(x)$, the first register collapses into a superposition of only those states $|x\rangle$ that could have produced the result $k$:
    $|\psi_2\rangle = \frac{1}{\sqrt{M}} \sum_{j=0}^{M-1} |x_0 + jr\rangle |k\rangle$
    where $x_0$ is the smallest input that gives the result $k$, and $M$ is the number of terms in the superposition. The state of the first register is now a uniform superposition of states separated by the period $r$.

4.  **Quantum Fourier Transform**: We apply the QFT to the first register. The QFT is a quantum analogue of the discrete Fourier transform, defined as:
    $QFT|x\rangle = \frac{1}{\sqrt{2^n}} \sum_{y=0}^{2^n-1} e^{2\pi i xy / 2^n} |y\rangle$
    Applying the QFT to $|\psi_2\rangle$ transforms the periodic superposition in the computational basis into a state where the probability amplitudes are sharply peaked at multiples of $2^n/r$.

5.  **Measure the First Register**: We measure the first register. The outcome will, with high probability, be a value $c$ that is an integer close to a multiple of $2^n/r$:
    $\frac{c}{2^n} \approx \frac{j}{r}$ for some integer $j$.

**Part 3: Classical Post-Processing**

The final step is to use the measurement result $c$ to find the period $r$.

1.  **Continued Fractions Algorithm**: We use the classical continued fractions algorithm on the value $c/2^n$ to find the best rational approximation $j/r$. Since we know the denominator $r$ must be less than $N$, the algorithm can efficiently find the fraction in lowest terms. The denominator of this fraction gives us our candidate for the period $r$.

2.  **Verification**: We verify if this $r$ is indeed the period by checking if $a^r \equiv 1 \pmod{N}$. If not, we can try other candidates from the continued fractions expansion or from other peaks in the QFT output. If we fail, we restart the quantum algorithm.

With a high probability of success in each run, the algorithm efficiently finds the factors of $N$.

---

### **Use Cases and Significance**

* **Breaking Cryptography**: The primary significance of Shor's algorithm is its ability to break widely used public-key cryptography systems. **RSA**, the most common public-key cryptosystem, relies on the classical difficulty of factoring large numbers. An adversary with a sufficiently large quantum computer could use Shor's algorithm to factor the public key and derive the private key, compromising secure communication. Similarly, related quantum algorithms can break other systems like **Diffie-Hellman** key exchange and **Elliptic Curve Cryptography**.

* **Driving Post-Quantum Cryptography**: The threat posed by Shor's algorithm has been a major driving force behind the development of **post-quantum cryptography (PQC)**. This is a field dedicated to creating new cryptographic algorithms that are secure against attacks from both classical and quantum computers.

* **A Benchmark for Quantum Computers**: Factoring has become a standard benchmark for measuring the progress of quantum hardware. Demonstrating the ability to factor increasingly large numbers is a key milestone on the roadmap to building a fault-tolerant universal quantum computer.

---

### **References**

* [82] Shor, P. W. (1994). Algorithms for Quantum Computation: Discrete Logarithms and Factoring. In *Proceedings of the 35th Annual Symposium on Foundations of Computer Science* (pp. 124–134). IEEE.
* [125] Shor, P. W. (1997). Polynomial-Time Algorithms for Prime Factorization and Discrete Logarithms on a Quantum Computer. *SIAM Journal on Computing*, 26(5), 1484–1509.
* Nielsen, M. A., & Chuang, I. L. (2010). *Quantum Computation and Quantum Information: 10th Anniversary Edition*. Cambridge University Press. (Provides a detailed textbook explanation in Chapter 5).